# Práctica final de Normativa y Regulación

El desarrollo consiste en utilizar dos dataset o conjunto, para hacer uso de datos abiertos(Open Data) de datos del catalogo de datos [Open Data Canarias](http://www.opendatacanarias.es/) y [TENERIFEDATA](http://www.tenerifedata.es) en concreto el conjunto de datos [alojamientos turisticos de tenerife](http://www.opendatacanarias.es/datos/dataset/3f637e7d-d875-48c8-9337-6f16f767c655/resource/a6864f3e-acda-40fd-92a1-ae292e713399/download/alojamientos.csv) y el conjunto de datos [paradas de guaguas de tenerife](http://www.tenerifedata.es/dataset/transporte/resource/8e0664aa-e67d-47e0-adec-54b37fd21e38)
*****

**Caracteristicas de estos conjuntos**
* **Paradas de guaguas de tenerife:**
    * **Resumen**: Recursos georeferenciados relacionados con la movilidad: aparcamientos, paradas de guaguas,...
    * **Última actualización:** desconocido
    * **Fecha de creación:** 2 Años
    * **Formatos:** JSON,CSV
    * **Licencia:** [Creative Commons CCZero](http://opendefinition.org/licenses/cc-zero/)
    * **Autor:** desconocido
    * **Mantenedor:** desconocido
    * **Versión:** desconocido
    * ** Cobertura Geográfica:** Tenerife
    * **Idioma:** español
* **Alojamientos turistico:**
    * **Resumen:** Contiene toda la información acerca de los alojamientos registrados por Turismo de Tenerife. La información que aporta este dataset es recogida desde el punto de vista de interés para el turista por lo que puede haber algún que no esté correcto. Los recursos se encuentran en los formatos: XML, JSON, CSV.<br>
    **NOTA:** Los ficheros en RDF se han realizado en colaboración con el grupo TARO de investigación de la Universidad de La Laguna.
    * **Última actualización:** Julio 3, 2015
    * ** Fecha de creación:** Junio 18,2014
    * **Formatos:** CSV, JSON, XML, RDF
    * **Licencia:** [Open Data Commons Attribution License](http://opendefinition.org/licenses/odc-by/)
    * **Autor:** SPET, Turismo de Tenerife S.A
    * **Mantenedor:** SPET, Turismo de Tenerife S.A
    * **Versión:** 1.1
    * ** Cobertura Geográfica:** Tenerife
    * **Idioma:** español  
    

## Aplicación
* ** Autor:** Orlandy Ariel Sánchez A.
* ** Asignatura:** Normativa y Regulación.
* ** Lenguaje: ** Python
* **Licencia:** [Creative Commons Reconocimiento 4.0 Internacional](https://creativecommons.org/licenses/by/4.0/)

Para el desarrollo de la aplicación se hará uso de datos abiertos(Open Data) utilizando la herramienta [Jupyter Notebook](http://jupyter.org/).
Se hará uso de las siguientes librerias:
* Para el texto se utilizará Markdown
* Para el código Python
* Se utilizarán las librerias:
    * [IPython](https://ipython.org/)
    * [gpxpy](https://pypi.python.org/pypi/gpxpy/0.7.0)
    * [Pandas](http://pandas.pydata.org/)
    * [Folium](https://folium.readthedocs.io/en/latest/)

### Paradas de guaguas de Tenerife


In [2]:
from IPython.display import HTML
import gpxpy.geo
import pandas as pd
import folium